<a href="https://colab.research.google.com/github/rawanhussein77/lecture-/blob/main/Welcome_To_Colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [8]:
import pandas as pd
import dash
from dash import dcc, html, Input, Output
import plotly.express as px

file_path = (r'/content/country_wise_latest (2).csv')
covid_data = pd.read_csv(file_path)


covid_data = covid_data.rename(columns={
    "Country/Region": "Country",
    "WHO Region": "Region"
})
covid_data["Active"] = covid_data["Confirmed"] - (covid_data["Deaths"] + covid_data["Recovered"])

app = dash.Dash(__name__, external_stylesheets=["https://codepen.io/chriddyp/pen/bWLwgP.css"])
app.title = "COVID-19 Dashboard"


app.layout = html.Div([
    html.Div([
        html.H1("COVID-19 Cases - Dashboard", style={"textAlign": "center", "margin-bottom": "20px"}),

        html.Div([
            html.Label("Select Date Range", style={"fontWeight": "bold"}),
            dcc.DatePickerRange(
                id="date-picker",
                start_date="2020-01-22",
                end_date="2020-03-11",
                display_format="YYYY-MM-DD",
                style={"margin-bottom": "20px"}
            )
        ]),

        html.Div([
            html.Label("Select Metric", style={"fontWeight": "bold"}),
            dcc.Dropdown(
                id="metric-selector",
                options=[
                    {"label": "Total Cases", "value": "Confirmed"},
                    {"label": "Recovered", "value": "Recovered"},
                    {"label": "Deaths", "value": "Deaths"},
                    {"label": "Active Cases", "value": "Active"}
                ],
                value="Confirmed",
                style={"width": "50%", "margin-bottom": "20px"}
            )
        ]),

        html.Div([
            html.Label("Select Country", style={"fontWeight": "bold"}),
            dcc.Dropdown(
                id="country-selector",
                options=[{"label": country, "value": country} for country in covid_data["Country"].unique()],
                placeholder="Select Country",
                multi=True,
                style={"width": "50%", "margin-bottom": "20px"}
            )
        ])
    ]),

    html.Div([
        html.Div([
            dcc.Graph(id="line-chart", style={"height": "400px"})
        ], style={"width": "100%", "display": "inline-block"}),

        html.Div([
            dcc.Graph(id="map-chart", style={"height": "400px"})
        ], style={"width": "100%", "display": "inline-block", "margin-top": "20px"}),

        html.Div([
            html.Div([
                dcc.Graph(id="bar-chart")
            ], style={"width": "48%", "display": "inline-block"}),

            html.Div([
                dcc.Graph(id="pie-chart")
            ], style={"width": "48%", "display": "inline-block"})
        ], style={"margin-top": "20px"})
    ])
])


@app.callback(
    [
        Output("line-chart", "figure"),
        Output("map-chart", "figure"),
        Output("bar-chart", "figure"),
        Output("pie-chart", "figure")
    ],
    [
        Input("date-picker", "start_date"),
        Input("date-picker", "end_date"),
        Input("metric-selector", "value"),
        Input("country-selector", "value")
    ]
)
def update_charts(start_date, end_date, selected_metric, selected_countries):

    filtered_data = covid_data

    if selected_countries:
        filtered_data = filtered_data[filtered_data["Country"].isin(selected_countries)]


    line_fig = px.line(
        filtered_data,
        x="Country",
        y=selected_metric,
        color="Region",
        title=f"{selected_metric} Trends by Country"
    )


    map_fig = px.scatter_geo(
        filtered_data,
        locations="Country",
        locationmode="country names",
        size=selected_metric,
        color="Region",
        title=f"{selected_metric} Distribution on Map",
        projection="natural earth"
    )


    bar_fig = px.bar(
        filtered_data,
        x="Country",
        y=selected_metric,
        color="Region",
        title=f"{selected_metric} by Country"
    )


    pie_fig = px.pie(
        filtered_data,
        values=selected_metric,
        names="Country",
        title=f"{selected_metric} Distribution by Country"
    )

    return line_fig, map_fig, bar_fig, pie_fig


if __name__ == "__main__":
    app.run_server(debug=True)


<IPython.core.display.Javascript object>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')